In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from sklearn.mixture import GaussianMixture
from tensorflow.keras.regularizers import l2

# Load datasets and initialize models
for n in [30]:
    # Load dataset
    import os
    print("Current Directory:", os.getcwd())
    Xs = np.load(f"/home/moritz/maths-for-ml/Kryptonite-N/Datasets/additional-kryptonite-{n}-X.npy")
    Ys = np.load(f"/home/moritz/maths-for-ml/Kryptonite-N/Datasets/additional-kryptonite-{n}-y.npy")
    df_x = pd.DataFrame(Xs)
    df_y = pd.Series(Ys)
    df_x_transformed = pd.DataFrame()

    X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)
    
    X_train_transformed = pd.DataFrame()
    X_test_transformed = pd.DataFrame()

    print(X_train.shape)
    print(X_test.shape)

    for column in df_x.columns:
        gmm = GaussianMixture(n_components=2, random_state=42)
        gmm.fit(X_train[[column]])
        proba = gmm.predict_proba(X_train[[column]])
        X_train_transformed[f'{column}_Mode_Prob'] = np.where(proba[:, 0] > proba[:, 1], -proba[:, 0], proba[:, 1])
        proba = gmm.predict_proba(X_test[[column]])
        X_test_transformed[f'{column}_Mode_Prob'] = np.where(proba[:, 0] > proba[:, 1], -proba[:, 0], proba[:, 1])
    
    print((X_train >= 0.5).astype(int).shape, X_train_transformed.shape)
    print((X_test >= 0.5).astype(int).shape, X_test_transformed.shape)
    X_train = pd.concat([(X_train >= 0.5).astype(int).reset_index(drop=True), X_train_transformed.reset_index(drop=True)], axis=1)
    X_test = pd.concat([(X_test >= 0.5).astype(int).reset_index(drop=True), X_test_transformed.reset_index(drop=True)], axis=1)

    print(X_train.shape)
    print(X_test.shape)
    # Split data
    
    # Build the neural network model
   

    model = Sequential([
    Input(shape=(X_train.shape[1],)),

    Dense(256),
    LeakyReLU(),
    BatchNormalization(),
    
    Dense(128),
    LeakyReLU(),
    BatchNormalization(),
    
    Dense(64),
    LeakyReLU(),
    BatchNormalization(),

    Dense(1, activation='sigmoid')
])


    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Define a callback to stop training when accuracy reaches 95%
    class EarlyStoppingByAccuracy(Callback):
        def on_epoch_end(self, epoch, logs=None):
            if logs.get('accuracy') >= 0.94:
                print("\nReached 94% accuracy, stopping training!")
                self.model.stop_training = True

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=150,
        batch_size=32,
        validation_split=0.1, 
        verbose=1,
        callbacks=[EarlyStoppingByAccuracy()]
    )

    # Make predictions and evaluate
    y_pred_nn = (model.predict(X_test) > 0.5).astype(int)
    accuracy_nn = accuracy_score(y_test, y_pred_nn)
    print(f"Accuracy of the Neural Network Classifier on test set for n = {n}: {accuracy_nn:.4f}")
    print(classification_report(y_test, y_pred_nn))


Current Directory: \\wsl.localhost\Ubuntu\home\moritz\maths-for-ml
(200000, 30)
(50000, 30)
(200000, 30) (200000, 30)
(50000, 30) (50000, 30)
(200000, 60)
(50000, 60)
Epoch 1/150
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.5015 - loss: 0.7129 - val_accuracy: 0.5046 - val_loss: 0.6965
Epoch 2/150
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 6s 996us/step - accuracy: 0.5028 - loss: 0.6947 - val_accuracy: 0.4963 - val_loss: 0.6966
Epoch 3/150
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.5016 - loss: 0.6940 - val_accuracy: 0.4951 - val_loss: 0.6938
Epoch 4/150
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.5012 - loss: 0.6936 - val_accuracy: 0.4956 - val_loss: 0.6935
Epoch 5/150
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 6s 974us/step - accuracy: 0.5046 - loss: 0.6933 - val_accuracy: 0.4982 - val_loss: 0.6936
Epoch 6/150
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 5s 958us/step - accuracy: 0.5014 - loss: 0.6934 - val_accuracy: 0.4958 - val_loss: 0.6937
Epoch 7/150
5625/5625 ━━━━━━━━━━━━━━━━━━━

In [13]:
# Save the model as moritz-n24
model.save('moritz-n24.keras')
